In [5]:
from data_generator.csv_data_processor import CSVDataProcessor
from utils.util_funcs import get_row_count, display_df, remove_data, verify_ranking, verify_ranking_counts
from utils.read_write import read_postgres_table
from spark_instance import spark
from pyspark.sql import Window

from pyspark.sql.functions import col, lower, lit, rand, array, floor, concat, when, row_number, concat_ws, date_format, count, split, size
  

/Users/jacobbickerstaff/Desktop/Development/health-data-etl-kaggle/venv/lib/python3.11/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(
24/02/22 23:11:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
# csv_reader = CSVDataProcessor(spark, "data/healthcare_dataset.csv")
# 
# # Read the CSV file
# raw_df = csv_reader.run()


In [7]:
# min_age_days = 1 * 365  # Minimum age in days (18 years)
# max_age_days = 90 * 365  # Maximum age in days (90 years)
# 
# raw_df = (raw_df.withColumn("DOB", expr(f"date_sub(current_date(), CAST(round(rand() * ({max_age_days} - {min_age_days}) + {min_age_days}) AS INT))"))
#         .withColumn("Age", floor(datediff(current_date(), col("DOB")) / 365.25)))




In [8]:
# above is commented out as it's saved in the database:
df = read_postgres_table("dob_age_raw_data")

In [9]:
# df.where(col("Age") < 16).show()

In [11]:
from constants.admission_types_dataset import admission_mapping, CONDITION_TESTS, CONDITIONS_CONST, STAY_TYPES


flattened = [
    (
        top_level.name,
        sub_level_key.name,
        stay_type,
        ", ".join([item for sublist in sub_level_info.get(CONDITIONS_CONST, [["No conditions"]]) for item in sublist]),  ", ".join(sub_level_info.get(CONDITION_TESTS, ["No tests"])) 
    )
    for top_level, sub_level_dict in admission_mapping.items()
    for sub_level_key, sub_level_info in sub_level_dict.items()
    for stay_type in (sub_level_info[STAY_TYPES] if isinstance(sub_level_info, dict) else sub_level_info)
]



In [12]:
flattened

[('EMERGENCY',
  'INJURY_RTC',
  'Inpatient',
  'Whiplash, Concussions, Contusions (bruises), Lacerations, Abrasions, Fractures, Sprains, Strains, Dislocations, Rib Fractures, Internal Bleeding, Pneumothorax (collapsed lung), Spinal Cord Injuries, Traumatic Brain Injuries (TBIs), Crush Injuries, Amputations, Psychological Trauma, Burn Injuries, Skull Fractures, Pelvic Fractures, Facial Fractures, Hematomas, Organ Damage, Cervical Spine Injuries, Thoracic Spine Injuries, Lumbar Spine Injuries, Tendon Injuries, Nerve Damage, Eye Injuries, Dental Injuries, Post-Traumatic Arthritis, Compartment Syndrome, Soft Tissue Injuries, Seat Belt Injuries, Airbag Injuries, Lower Extremity Injuries, Upper Extremity Injuries, Drowning or Near-Drowning Incidents, Hypothermia or Frostbite, Heatstroke, Acoustic Trauma, Stress Fractures, Bursitis',
  'X-rays, CT scans, MRI, Ultrasound, Blood tests'),
 ('EMERGENCY',
  'INJURY_RTC',
  'Day Patient',
  'Whiplash, Concussions, Contusions (bruises), Lacerations

In [13]:
"""
STEP 1 
    From the flattened data list we wish to create a DataFrame. This contains all the possible combinations for the given 
    top level admissions, sub level admissions, stay types and list of tests available from the admission_mapping, stay_type and admission_tests lists or dictionaries in admission_types_test_dataset.py
"""
mapping_df = spark.createDataFrame(flattened, ["top_level_admission", "sub_level_admission", "stay_type", "possible_tests", "conditions"])

display_df(mapping_df, 45)

,top_level_admission,sub_level_admission,stay_type,possible_tests,conditions
0,EMERGENCY,INJURY_RTC,Inpatient,"Whiplash, Concussions, Contusions (bruises), L...","X-rays, CT scans, MRI, Ultrasound, Blood tests"
1,EMERGENCY,INJURY_RTC,Day Patient,"Whiplash, Concussions, Contusions (bruises), L...","X-rays, CT scans, MRI, Ultrasound, Blood tests"
2,EMERGENCY,SELF_INFLICTED,Inpatient,No conditions,"Psychological assessment, X-rays (for physical..."
3,EMERGENCY,CARDIOLOGY,Inpatient,"Hypertension (High Blood Pressure), Coronary A...","ECG, Echocardiogram, Stress tests, Cardiac cat..."
4,EMERGENCY,CARDIOLOGY,Day Patient,"Hypertension (High Blood Pressure), Coronary A...","ECG, Echocardiogram, Stress tests, Cardiac cat..."
5,EMERGENCY,NEUROLOGY,Inpatient,"Migraines, Epilepsy, Parkinson's Disease, Alzh...","MRI or CT scans of the brain, Electroencephalo..."
6,EMERGENCY,NEUROLOGY,Day Patient,"Migraines, Epilepsy, Parkinson's Disease, Alzh...","MRI or CT scans of the brain, Electroencephalo..."
7,EMERGENCY,NEUROLOGY,Outpatient,"Migraines, Epilepsy, Parkinson's Disease, Alzh...","MRI or CT scans of the brain, Electroencephalo..."
8,EMERGENCY,GASTROENTEROLOGY,Inpatient,"Gastro-oesophageal Reflux Disease (GORD), Irri...","Endoscopy, Colonoscopy, Blood tests, Stool tes..."
9,EMERGENCY,GASTROENTEROLOGY,Day Patient,"Gastro-oesophageal Reflux Disease (GORD), Irri...","Endoscopy, Colonoscopy, Blood tests, Stool tes..."


In [9]:
"""
STEP 2 
    Create a list of admission_types randomly assign this to the original patient in the original data set, 
    whilst dropping the original admission_type column. Then join with mapping_df on top_level_admission col to give access to possible 
    conditions, mappings and so on.
"""
admission_types = list(admission_mapping.keys())

print(admission_types)

keys_array = array([lit(key) for key in admission_types])

# Define constants and conditions
female_only = ['maternity', 'obstetrics']
is_female = lower(col('gender')) == 'female'
is_pediatric = col("Age") < 18
is_geriatric = (col("Age") >= 65) & (col("sub_level_admission") == "geriatrics")

df = (df.withColumn("top_level_admission", keys_array[floor(rand() * len(admission_types))]).drop("admission_type")
        .withColumn("is_female", is_female)
        .withColumn("is_pediatric", is_pediatric)
      )
get_row_count(df, True)
df = df.join(mapping_df, "top_level_admission", "inner")
get_row_count(df, True)
display_df(df, 999)

['emergency', 'gp_referral', 'hospital_referral', 'self_referral', 'elective']
10000
90137


,top_level_admission,name,Age,gender,blood_type,medical_condition,date_of_admission,doctor,hospital,insurance_provider,...,room_number,discharge_date,medication,test_results,DOB,is_female,is_pediatric,sub_level_admission,stay_type,possible_tests
0,elective,Chad Byrd,59,Male,B-,Obesity,2019-01-09,Paul Baker,Walton LLC,Medicare,...,292,2019-02-08,Lipitor,Normal,1964-06-30,False,False,maternity,day_patient,"[Ultrasound, Blood tests, Glucose tolerance te..."
1,elective,Chad Byrd,59,Male,B-,Obesity,2019-01-09,Paul Baker,Walton LLC,Medicare,...,292,2019-02-08,Lipitor,Normal,1964-06-30,False,False,maternity,inpatient,"[Ultrasound, Blood tests, Glucose tolerance te..."
2,elective,Chad Byrd,59,Male,B-,Obesity,2019-01-09,Paul Baker,Walton LLC,Medicare,...,292,2019-02-08,Lipitor,Normal,1964-06-30,False,False,orthopedics,day_patient,"[X-rays, MRI, CT scans, Bone scans, Blood tests]"
3,elective,Chad Byrd,59,Male,B-,Obesity,2019-01-09,Paul Baker,Walton LLC,Medicare,...,292,2019-02-08,Lipitor,Normal,1964-06-30,False,False,orthopedics,inpatient,"[X-rays, MRI, CT scans, Bone scans, Blood tests]"
4,elective,Chad Byrd,59,Male,B-,Obesity,2019-01-09,Paul Baker,Walton LLC,Medicare,...,292,2019-02-08,Lipitor,Normal,1964-06-30,False,False,obstetrics,inpatient,"[Ultrasound, Blood tests, Glucose tolerance te..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,elective,Daniel Mccoy,24,Female,AB+,Asthma,2022-12-25,Dylan Andrews,Sanchez Ltd,UnitedHealthcare,...,250,2022-12-30,Penicillin,Inconclusive,2000-02-04,True,False,obstetrics,inpatient,"[Ultrasound, Blood tests, Glucose tolerance te..."
995,elective,Daniel Mccoy,24,Female,AB+,Asthma,2022-12-25,Dylan Andrews,Sanchez Ltd,UnitedHealthcare,...,250,2022-12-30,Penicillin,Inconclusive,2000-02-04,True,False,ophthalmology,day_patient,"[Eye exam, Tonometry (eye pressure test), Reti..."
996,elective,Daniel Mccoy,24,Female,AB+,Asthma,2022-12-25,Dylan Andrews,Sanchez Ltd,UnitedHealthcare,...,250,2022-12-30,Penicillin,Inconclusive,2000-02-04,True,False,ophthalmology,outpatient,"[Eye exam, Tonometry (eye pressure test), Reti..."
997,elective,Daniel Mccoy,24,Female,AB+,Asthma,2022-12-25,Dylan Andrews,Sanchez Ltd,UnitedHealthcare,...,250,2022-12-30,Penicillin,Inconclusive,2000-02-04,True,False,geriatrics,inpatient,"[Comprehensive geriatric assessment, Blood tes..."


In [10]:
"""
STEP 3 
    Create a rowq ranking using a unique cols, created from stay_name and unique_id. 
    Verification done below. 
    
"""

df =  df.withColumn('stay_name', 
                   when(col('stay_type') == 'out_patient', concat(col('name'), lit('_out_patient')))
                   .when(col('stay_type') == 'inpatient', concat(col('name'), lit('_inpatient')))
                   .when(col('stay_type') == 'day_patient', concat(col('name'), lit('_day_patient')))
                   .otherwise(col('name'))
                  ).withColumn("unique_id", concat_ws("_", "name", date_format("DOB", "yyyyMMdd")))


# Define a window specification that partitions data by 'top_level_admission' (or another unique patient identifier if needed)
windowSpec = Window.partitionBy('stay_name', 'unique_id').orderBy(rand())

# Assign row numbers within each partition in a random order
ranked_df = df.withColumn("row_num", row_number().over(windowSpec))


In [11]:
ranked_df.createGlobalTempView("ranked_df")

unique_dobs_df = spark.sql("""
WITH NameCounts AS (
    SELECT name
    FROM global_temp.ranked_df
)

SELECT DISTINCT r.name, r.unique_id
FROM global_temp.ranked_df r
JOIN NameCounts n ON r.name = n.name
ORDER BY r.name
LIMIT 10
""")

24/02/13 20:31:02 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [12]:
# COSTLY WAY TO VERIFY THAT THE RANK WORKS BELOW....  due to .collect() and .count() in verify_ranking_counts() function

# Example usage:
unique_names = [row['name'] for row in unique_dobs_df.select("name").collect()]
unique_ids = [row['unique_id'] for row in unique_dobs_df.select("unique_id").collect()]
verify_ranking_counts(df, ranked_df, unique_names, unique_ids)

# this function is faster than above
verify_ranking(df, ranked_df)


In [13]:
"""
Step 4: 
    Sort out geriatrics data and verify that individuals are not geriatrics 
"""
ranked_df = ranked_df.withColumn("is_geriatric", is_geriatric)

not_geriatric_df = ranked_df.where((col('sub_level_admission') == 'geriatrics') &( col("is_geriatric") == False))

filtered_df = remove_data(ranked_df, not_geriatric_df, (col('sub_level_admission') == 'geriatrics') ,  ( col("is_geriatric") == False))


In [14]:
not_female_df = filtered_df.where((col("sub_level_admission").isin(female_only)) & (col("is_female") == False))
filtered_df_female = remove_data(filtered_df, not_female_df, (col("gender") == "Male"),  (col("sub_level_admission").isin(female_only)))


In [15]:
# filter pediatric patients who cannot be pregnant (based on legal age in the UK, 16) No assumption made an individual cannot choose to get pregnant before this age. 
under16_female_df = filtered_df_female.where((col("Age") <16) & (col("sub_level_admission") == 'maternity')).orderBy("Age")

In [16]:
df = remove_data(filtered_df_female, under16_female_df, (col("sub_level_admission") == "maternity"), (col("Age") <16))

In [17]:

split_col = split(df['name'], ' ')

df_renamed = (df.withColumn('name', 
                    when(size(split_col) == 4, concat(split_col.getItem(1).substr(1, 1), lit('. '), split_col.getItem(2).substr(1, 1), lit('. '), split_col.getItem(3)))
                    .when(size(split_col) == 3, concat(split_col.getItem(1).substr(1, 1), lit('. '), split_col.getItem(2)))
                    .otherwise(
                        concat(split_col.getItem(0).substr(1, 1), lit('. '), split_col.getItem(1))
                               )
                    )
      )


In [21]:
df_renamed.orderBy('name', 'Age').show(n=200)

+--------------------+----------+-------------------+------------+---+------+----------+-----------------+-----------------+-------------------+--------------------+------------------+------------------+-----------+--------------+-----------+------------+---------+------------+-----------+--------------------+--------------------+--------------------+-------+------------+
| sub_level_admission|       DOB|top_level_admission|        name|Age|gender|blood_type|medical_condition|date_of_admission|             doctor|            hospital|insurance_provider|    billing_amount|room_number|discharge_date| medication|test_results|is_female|is_pediatric|  stay_type|      possible_tests|           stay_name|           unique_id|row_num|is_geriatric|
+--------------------+----------+-------------------+------------+---+------+----------+-----------------+-----------------+-------------------+--------------------+------------------+------------------+-----------+--------------+-----------+--------

In [ ]:
df_renamed.select(['name', 're_name']).show(n=2000)

In [ ]:
# TODO: drop full name for initial and surname since nname column is poor. 
# adjust doctor column 
# medical condition 
# admission date to be checked again dob, 
# medication to be checked 
# choose random results 
# TODO filter on is pediatric, geriatric and is_female to be done here and same people with dob? needs  to be considered 
# drop stay_name and unique_id 


In [ ]:
# Filter to keep only the top-ranked row within each partition
ranked_df = filtered_df_female

In [ ]:
ranked_df.where(col("name") == "Daniel Mccoy").show(n=8000)

In [ ]:
from pyspark.sql.types import StringType

df = filtered_df_female.select([col(c).cast(StringType()).alias(c) for c in df.columns])
df.write.csv('./temp_data/female/filtered_df_female.csv', mode = 'overwrite', header=True)

In [ ]:
# TODO initialise first name to make data more realistic for name columns i.e. Daniel Mccoy is seen a Male name but here its Female

In [ ]:
spark.stop()

In [5]:
dict_ = admission_mapping.get("emergency")